In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
import yfinance as yf

In [2]:
# get sp500 tickers from wikipedia
wiki_sp500_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500_table = pd.read_html(wiki_sp500_url)[0].sort_values('Symbol')
sp500_table.reset_index(inplace = True, drop = True)
sp500_table.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,A,Agilent Technologies,Health Care,Health Care Equipment,"Santa Clara, California",2000-06-05,1090872,1999
1,AAL,American Airlines Group,Industrials,Passenger Airlines,"Fort Worth, Texas",2015-03-23,6201,1934
2,AAP,Advance Auto Parts,Consumer Discretionary,Automotive Retail,"Raleigh, North Carolina",2015-07-09,1158449,1932
3,AAPL,Apple Inc.,Information Technology,"Technology Hardware, Storage & Peripherals","Cupertino, California",1982-11-30,320193,1977
4,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)


In [3]:
start_date = "2016-01-01"

# download data for the sp500 tickers
data = yf.download(list(sp500_table['Symbol']))
data = data.loc[start_date:]
data = data["Adj Close"]

data.head()

[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
- BF.B: 1d data not available for startTime=-2208994789 and endTime=1685803939. Only 100 years worth of day granularity data are allowed to be fetched per request.
- BRK.B: No timezone found, symbol may be delisted


,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,38.349049,39.095463,140.835495,24.041487,41.642838,90.170334,37.339104,22.950001,90.301270,91.970001,...,62.144653,28.846169,54.520844,55.388367,32.865227,45.203148,93.360924,66.489998,22.265125,44.926643
2016-01-05,38.217102,38.722755,139.873428,23.439028,41.469353,91.489235,37.330395,23.033333,90.771278,92.339996,...,63.319408,29.137056,54.985386,56.517601,32.856113,45.090466,95.304977,64.820000,22.023386,45.629948
2016-01-06,38.386730,39.401268,136.173019,22.980330,41.476582,90.090698,37.017288,23.070000,90.593925,91.019997,...,60.039131,29.444092,54.527874,55.698910,32.446213,44.771210,95.717644,62.230000,21.381525,45.639446
2016-01-07,36.756279,38.655857,137.680923,22.010452,41.353710,87.213951,36.130131,23.046667,87.933578,89.110001,...,54.391262,29.557224,53.655090,55.209576,31.608189,43.243774,93.516800,59.410000,20.739664,44.251827
2016-01-08,36.369858,38.579403,134.655884,22.126837,40.226074,86.709396,35.373425,22.806667,87.082245,87.849998,...,52.177296,29.234016,52.571144,53.628670,31.307587,42.655346,93.122498,59.250000,20.506260,43.605537


In [4]:
# create a new dataframe
cols = ['Latest Price', 
        'One-Year Price Return', 
        'One-Year Return Percentile', 
        'Six-Month Price Return', 
        'Six-Month Return Percentile', 
        'Three-Month Price Return', 
        'Three-Month Return Percentile', 
        'One-Month Price Return', 
        'One-Month Return Percentile',
        'HQM Score',
        'Number of Shares to Buy']
df = pd.DataFrame(columns = cols)

# calculate price returns
df["Latest Price"] = data.iloc[-1]
df["One-Year Price Return"] = (data.iloc[-1] / data.iloc[-252] - 1)
df["Six-Month Price Return"] = (data.iloc[-1] / data.iloc[-126] - 1)
df["Three-Month Price Return"] = (data.iloc[-1] / data.iloc[-63] - 1)
df["One-Month Price Return"] = (data.iloc[-1] / data.iloc[-21] - 1)

df.rename_axis(index = 'Ticker', inplace =  True)
df.reset_index(inplace = True)
df.head()

,Ticker,Latest Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score,Number of Shares to Buy
0,A,118.220001,-0.078197,NaN,-0.240397,NaN,-0.173270,NaN,-0.116970,NaN,NaN,NaN
1,AAL,14.870000,-0.148339,NaN,0.063662,NaN,-0.077543,NaN,0.100666,NaN,NaN,NaN
2,AAP,67.559998,-0.631816,NaN,-0.543104,NaN,-0.493779,NaN,-0.438124,NaN,NaN,NaN
3,AAPL,180.949997,0.203826,NaN,0.223633,NaN,0.177925,NaN,0.092951,NaN,NaN,NaN
4,ABBV,136.869995,-0.031150,NaN,-0.137275,NaN,-0.110451,NaN,-0.071186,NaN,NaN,NaN


In [5]:
# for the given time periods compute return percentiles
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

def percentile_score(data, value):
    score = sum(1 for x in data if x <= value) / len(data)
    return score


for row in df.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        s = percentile_score(df[change_col], df.loc[row, change_col])
        df.loc[row, percentile_col] = s

df.head()

,Ticker,Latest Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score,Number of Shares to Buy
0,A,118.220001,-0.078197,0.397614,-0.240397,0.069583,-0.173270,0.091451,-0.116970,0.035785,NaN,NaN
1,AAL,14.870000,-0.148339,0.264414,0.063662,0.743539,-0.077543,0.304175,0.100666,0.827038,NaN,NaN
2,AAP,67.559998,-0.631816,0.003976,-0.543104,0.001988,-0.493779,0.001988,-0.438124,0.001988,NaN,NaN
3,AAPL,180.949997,0.203826,0.833002,0.223633,0.912525,0.177925,0.934394,0.092951,0.803181,NaN,NaN
4,ABBV,136.869995,-0.031150,0.479125,-0.137275,0.212724,-0.110451,0.198807,-0.071186,0.089463,NaN,NaN


In [6]:
# calculate the high quality momentum (HQM) scores
for row in df.index:
    momentum_precentiles = [df.loc[row, f'{time_period} Return Percentile'] for time_period in time_periods]
    df.loc[row, 'HQM Score'] = np.mean(momentum_precentiles)       

df.head()

,Ticker,Latest Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score,Number of Shares to Buy
0,A,118.220001,-0.078197,0.397614,-0.240397,0.069583,-0.173270,0.091451,-0.116970,0.035785,0.148608,NaN
1,AAL,14.870000,-0.148339,0.264414,0.063662,0.743539,-0.077543,0.304175,0.100666,0.827038,0.534791,NaN
2,AAP,67.559998,-0.631816,0.003976,-0.543104,0.001988,-0.493779,0.001988,-0.438124,0.001988,0.002485,NaN
3,AAPL,180.949997,0.203826,0.833002,0.223633,0.912525,0.177925,0.934394,0.092951,0.803181,0.870775,NaN
4,ABBV,136.869995,-0.031150,0.479125,-0.137275,0.212724,-0.110451,0.198807,-0.071186,0.089463,0.24503,NaN


In [7]:
# select top 50 stocks
df.sort_values('HQM Score', ascending = False, inplace = True)
df = df[:50]
df.head()

,Ticker,Latest Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score,Number of Shares to Buy
340,NVDA,393.269989,1.009175,0.992048,1.295517,0.994036,0.669936,0.996024,0.426856,0.994036,0.994036,NaN
47,AVGO,812.000000,0.453822,0.950298,0.497355,0.984095,0.292177,0.976143,0.330799,0.99006,0.975149,NaN
330,NFLX,400.470001,0.952655,0.99006,0.263512,0.936382,0.283434,0.974155,0.248426,0.976143,0.969185,NaN
300,META,272.609985,0.370864,0.934394,1.263451,0.992048,0.474364,0.994036,0.167395,0.930417,0.962724,NaN
390,RCL,86.230003,0.493160,0.960239,0.415695,0.976143,0.162756,0.930417,0.199638,0.952286,0.954771,NaN


In [8]:
# get the size of the portfolio
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio: ')
    
    try:
        float(portfolio_size)
    except:
        print('That is not a number\nPlease try again: ')
        portfolio_size = input('Enter the size of your portfolio: ')
        
portfolio_input()

Enter the size of your portfolio: 1000000


In [9]:
# compute the number of shares
position_size = float(portfolio_size)/len(df.index) 
for i in df.index:
    df.loc[i,'Number of Shares to Buy'] = math.floor(position_size/df['Latest Price'][i])

In [10]:
df.head()

,Ticker,Latest Price,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score,Number of Shares to Buy
340,NVDA,393.269989,1.009175,0.992048,1.295517,0.994036,0.669936,0.996024,0.426856,0.994036,0.994036,50
47,AVGO,812.000000,0.453822,0.950298,0.497355,0.984095,0.292177,0.976143,0.330799,0.99006,0.975149,24
330,NFLX,400.470001,0.952655,0.99006,0.263512,0.936382,0.283434,0.974155,0.248426,0.976143,0.969185,49
300,META,272.609985,0.370864,0.934394,1.263451,0.992048,0.474364,0.994036,0.167395,0.930417,0.962724,73
390,RCL,86.230003,0.493160,0.960239,0.415695,0.976143,0.162756,0.930417,0.199638,0.952286,0.954771,231


In [11]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine = 'xlsxwriter')
df.to_excel(writer, 'Momentum Strategy', index = False)

In [12]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [13]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Latest Price', dollar_template],
    'C': ['One-Year Price Return', percent_template],
    'D': ['One-Year Return Percentile', percent_template],
    'E': ['Six-Month Price Return', percent_template],
    'F': ['Six-Month Return Percentile', percent_template],
    'G': ['Three-Month Price Return', percent_template],
    'H': ['Three-Month Return Percentile', percent_template],
    'I': ['One-Month Price Return', percent_template],
    'J': ['One-Month Return Percentile', percent_template],
    'K': ['HQM Score',percent_template],
    'L': ['Number of Shares to Buy', integer_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [14]:
writer.save()